In [ ]:
!pip install -r requirements.txt

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

%matplotlib inline

## Inizializzazione del dataset

Il dataset è composto da due sotto file in formato .csv:
1. trainingData, è un dataframe pandas creato a partire dal file di training "train.csv"
2. testData, è un dataframe pandas creato a partire dal file di testing "test.csv"


In [ ]:
trainData = pd.read_csv('train.csv')
testData = pd.read_csv('test.csv')

In [ ]:
trainData.head()

In [ ]:
testData.head()

## Controllo iniziale del TrainData set

Una volta caricato il TrainData set controllo che i dati presenti all'interno siano corretti, ovvero che non vi siano campi nulli o NaN. In questo caso la ricerca dei duplicati è futile poiché non abbiamo conoscenza sull'identità del paziente. 

In [ ]:
missing_data = trainData.isnull()
for column in trainData:
    print(column)
    print(missing_data[column].value_counts())
    print('')

In [ ]:
NaN_data = trainData.isna()
for column in trainData:
    print(column)
    print(NaN_data[column].value_counts())
    print('')

In [ ]:
trainData.dropna(inplace=True)

## Controllo iniziale del TestData set

Una volta caricato il TrainData set controllo che i dati presenti all'interno siano corretti, ovvero che non vi siano campi nulli o NaN. In questo caso la ricerca dei duplicati è futile poiché non abbiamo conoscenza sull'identità del paziente. 

In [ ]:
missing_data = testData.isnull()
for column in testData:
    print(column)
    print(missing_data[column].value_counts())
    print('')

è necessario eliminare le righe dove vi sono dei valori NaN poiché non possiamo aggiungere dati manualmente, potremmo creare delle discrepanze che la ridotta mole di dati non compenserebbe. 

In [ ]:
NaN_data = testData.isna()
for column in testData:
    print(column)
    print(NaN_data[column].value_counts())
    print('')

In [ ]:
testData.dropna(inplace=True)

## Controllo dettagliato del trainData set

Una volta controllata la presenza di valori nulli o duplicati all'interno del dataset è possibile analizzarne nel dettaglio le varie colonne. Questa analisi supporta la successiva operazione di feature selection, che verrà effettuata analizzando sia i singoli grafici DEATH_EVENT - feature e sia la correlation map. 

In [ ]:
for i, col in enumerate(trainData.columns):
    plt.figure(i)
    sns.lineplot(x='DEATH_EVENT',y=col, data=trainData)

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(trainData.corr(), annot=True, fmt='.1g');

In [ ]:
correlation = trainData.corr()
correlation_target = abs(correlation["DEATH_EVENT"])

Scegliamo di filtrare i nostri dati per ottenere solo le features più correlate con la feature target. Per questo viene impostata una soglia di correlazione minima pari a 0.1, in questo modo tutte le feature a correlazione molto bassa vengono scartate poiché non interessanti. 

In [ ]:
features = correlation_target[correlation_target > 0.1]
print(features)

L'analisi della heatmap e della corrleazione tra le varie condizioni mediche del paziente fanno emergere un forte collegamento tra la possibile morte durante il periodo di osservazione con le seguenti caratteristiche:
1. L'età del paziente
2. La Ejection Fraction, ovvero la frazione di eiezione è la frazione volumetrica del fluido espulsa da una camera ad ogni contrazione, un valore normale è compreso tra 50 e 75 
3. la concentrazione nel siero di creatina e sodio
4. Il tempo di osservazione, più il paziente è sotto osservazione e più è probabile che sia posssibile tenere traccia di un possibile un arresto cardiaco

L'analisi dei grafici valida questa decisione, inoltre si nota come la feature "smoking" e "sex" potrebbero essere correlate tra loro, ovvero che l'incidenza di arresti cardiaci potrebbe anche essere collegata al binomio genere-fumatore.

Istanzio una struttura dati chiamata selectedFeatures all'interno della quale inserisco le label delle feature scelte in seguito all'analisi dei dati.

In [ ]:
selected_features = ["age", "creatinine_phosphokinase", "ejection_fraction", "serum_creatinine", "serum_sodium", "time"]

## Inizializzazione Modello: RandomForest

Il modello scelto è un classificatore RandomForest. Il classificatore RandomForest utilizza diversi classificatori Decision Tree su diversi sotto campioni del dataset di input. Ogni classificatore compierà un'operazione di fitting sul suo sottocampione. Il classificatore RandomForest utilizzerà una funzione di averaging, ovvero fa la media tra le varie predizioni, per migliorare l'accuratezza e tenere sotto controllo l'overfitting. 

Un Decision tree classifier ha il compito di predirre il valore della variabile target, nel nostro caso DEATH_EVENT, apprendendo delle regole di decisione a partire dalle feature del nostro dataset

Il classificatore viene poi valutato in accuratezza, tramite la metrica accuracy_score, sia sul dataset di training che su quello di test.

In [ ]:
datasetTrain = trainData[selected_features]

In [ ]:
datasetTest = testData[selected_features]

In [ ]:
y_train = trainData["DEATH_EVENT"]

In [ ]:
y_test = testData["DEATH_EVENT"]

In [ ]:
clf = RandomForestClassifier(max_depth=6, n_estimators=32, random_state=42)

In [ ]:
clf.fit(datasetTrain, y_train)

In [ ]:
predict_TrainingSet = clf.predict(datasetTrain)

In [ ]:
predict_TestSet = clf.predict(datasetTest)

### Accuratezza

L'accuratezza viene calcolata usando le previsioni prodotte dal modello sia sul trainingSet che sul testSet.

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
print("Training set accuracy: {:.2f}".format(clf.score(datasetTrain, y_train)))

In [ ]:
print("Test set accuracy: {:.2f}".format(accuracy_score(y_test, predict_TestSet)))

In [ ]:
sns.heatmap(confusion_matrix(y_test, predict_TestSet),annot=True)
plt.ylabel("Actual")
plt.xlabel("Prediction")

## Conclusioni

Il nostro dataset è caratterizzato da una ridotta quantità di dati, questo comporta un rischio di overfitting che potrebbe inficiare le prestazioni del nostro modello. Per controllarne la validità possiamo effettuare un controllo sulla sua accuratezza sfruttando la K-Fold Cross Validation, nello specifico la 10-Fold Cross Validation.

Inoltre è da notare come il dataset sia sbilanciato verso i dati relativi alle persone che non hanno subito arresti cardiaci, ovvero la presenza di dati relativi ai decessi è molto esigua e questo potrebbe inficiare sul processo di addestramento. 

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
scores = cross_val_score(clf, datasetTest, y_test, cv=10)

In [ ]:
print("Accuracy using K-Fold Cross Validation, with k=10 : %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))